In [ ]:
# UNet++: A Nested U-Net Architecture for Medical Image Segmentation (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7329239/)
# This code follows the tructure from the paper 
# Properly working

import glob
import os
import cv2
import imageio
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import gc
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, Dropout, Input, BatchNormalization
from keras import optimizers
from keras.models import Model
from keras.losses import binary_crossentropy
from contextlib import redirect_stdout
import time
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from keras.models import load_model


#Start Execution Time
start_time = time.time()

path_out = "..."
path_in_tumor = "..."
path_in_mask = "..."


if not os.path.exists(path_out):
    os.makedirs(path_out)
    print("Directory Created ")
else:    
    print("Directory already exists") 

# Downsamples (Resizes) image to target size
def downsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_target, img_size_target), mode='constant', preserve_range=True,)

# Upsamples (Resizes) image to target size
def upsample(img):
    if img_size_ori == img_size_target:
        return img
    return resize(img, (img_size_ori, img_size_ori), mode='constant', preserve_range=True)

# Function for a Convolution layer containing 2 blocks
def conv2d_block( input_tensor, n_filters, kernel_size = (3,3), name="contraction"):
  "Add 2 conv layer"
  x = Conv2D(filters=n_filters, kernel_size=kernel_size, kernel_initializer='he_normal', 
             padding='same',activation="relu", name=name+'_1')(input_tensor)
  
  x = Conv2D(filters=n_filters, kernel_size=kernel_size, kernel_initializer='he_normal', 
             padding='same',activation="relu",name=name+'_2')(x)
  return x

def dice_coef(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    smooth = 1
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def bce_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred)

def loss_func(y_true, y_pred):
    d_loss = dice_coef_loss(y_true, y_pred)
    b_loss = bce_loss(y_true, y_pred)
    return (0.5*d_loss)+(0.5*b_loss)

images = []
masks = []

files_images = glob.glob (path_in_tumor + "*.jpg") 
for myFile in files_images:
    im = cv2.imread (myFile, cv2.IMREAD_GRAYSCALE)
    im = cv2.resize(im, (512, 512))
    #im = im/127.5 - 1
    images.append (im)
    
files_masks = glob.glob (path_in_mask + "*.jpg") 
for myFile in files_masks:
    im1 = cv2.imread (myFile, cv2.IMREAD_GRAYSCALE)
    im1 = cv2.resize(im1, (512, 512))
    #im1 = im1/127.5 - 1
    masks.append (im1)

images = np.array(images, dtype= 'float32') 
images = np.clip((images/12728),0,1)
masks = np.array(masks, dtype= 'float32') * 1

print(images.shape)
print(masks.shape)


img_size_ori = 512
img_size_target = 128


# Expands image to 3D by adding 1 as channel
images = np.expand_dims(images,axis=-1)
masks = np.expand_dims(masks,axis=-1)

# Reshaping to 128 x 128
images = np.array([ downsample(image) for image in images ])
masks = (np.array([ downsample(mask) for mask in masks ])>0)*1
print(images.shape)
print(masks.shape)


# 18 random input images with tumor highlighted
plt.figure(figsize=(12, 5))
for i, idx in enumerate(np.random.randint(images.shape[0], size=18), start=1):
    plt.subplot(3, 6, i)
    plt.imshow(images[idx], cmap='gray')
    plt.imshow(np.ones_like(masks[idx])-masks[idx], alpha=0.5, cmap='Set1')
    plt.title('MRI')
    plt.axis('off')
plt.savefig(path_out + 'Sample_Image_Plot.jpg')

# 9 random input tumor images and their corresponding masks
plt.figure(figsize=(12, 5))
i=1
for idx in np.random.randint( images.shape[0], size=9):
  plt.subplot(3,6,i);i+=1
  plt.imshow(images[idx])
  plt.title("Train Image")
  plt.axis('off')
  plt.subplot(3,6,i);i+=1
  plt.imshow(masks[idx]) 
  plt.title("Train Mask")
  plt.axis('off')
plt.savefig(path_out + 'Sample_Image_Mask_Plot.jpg')

# Training Test division
X_train,X_test,Y_train,Y_test = train_test_split( images,masks,test_size=0.2)
del images
del masks

gc.collect()
X_train.shape,X_test.shape
tr_data = str(Y_train.shape)
ts_data = str(Y_test.shape)

# Training is divided into training and validation
X_train1,X_val,Y_train1,Y_val = train_test_split(X_train,Y_train,test_size=0.2)

del X_train
del Y_train

gc.collect()
X_train1.shape,X_val.shape

file0 = open(path_out + 'Data_division.txt','w')
file0.write("Initial Training Data: " + str(tr_data)) 
file0.write("\n")
file0.write("Test Data: " + str(ts_data)) 
file0.write("\n")
file0.write("Final Training Data: " + str(Y_train1.shape)) 
file0.write("\n")
file0.write("Validation Data: " + str(Y_val.shape)) 
file0.close()


# Flips array in the left/right direction.
X_train1 = np.append( X_train1, [ np.fliplr(x) for x in X_train1], axis=0 )
Y_train1 = np.append( Y_train1, [ np.fliplr(y) for y in Y_train1], axis=0 )
X_train1.shape,Y_train1.shape

# Converting train and validation data with image data generator
train_datagen = ImageDataGenerator(brightness_range=(0.9,1.1),
                                   zoom_range=[.9,1.1],
                                   fill_mode='nearest')
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(X_train1, Y_train1, batch_size=8)
val_generator = val_datagen.flow(X_val, Y_val, batch_size=8)


# Initial input with initial assigned shape
IMG_DIM = (128,128,1)

inp = Input( shape=IMG_DIM )

# ------------------------------------------------------------------------
# X(0,0)
d1 = conv2d_block( inp, 64, name="contraction_1")
p1 = MaxPooling2D( pool_size=(2,2), strides=(2,2))(d1)
p1 = BatchNormalization(momentum=0.8)(p1)
p1 = Dropout(0.1)(p1)

# X(1,0)
d2 = conv2d_block( p1, 128, name="contraction_2_1" )
p2 = MaxPooling2D(pool_size=(2,2), strides=(2,2) )(d2)
p2 = BatchNormalization(momentum=0.8)(p2)
p2 = Dropout(0.1)(p2)

# X(2,0)
d3 = conv2d_block( p2, 256, name="contraction_3_1")
p3 = MaxPooling2D(pool_size=(2,2), strides=(2,2) )(d3)
p3 = BatchNormalization(momentum=0.8)(p3)
p3 = Dropout(0.1)(p3)

# X(3,0)
d4 = conv2d_block(p3,512, name="contraction_4_1")
p4 = MaxPooling2D(pool_size=(2,2), strides=(2,2) )(d4)
p4 = BatchNormalization(momentum=0.8)(p4)
p4 = Dropout(0.1)(p4)

# X(4,0)
d5 = conv2d_block(p4,512, name="contraction_5_1")


# ------------------------------------------------------------------------
# ---------------- UNet++ L1 ---------------
# X(0,1)
du_01 = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same')(d2)
du_01 = concatenate([du_01,d1])
du_01 = Dropout(0.1)(du_01)
c_01 = conv2d_block(du_01, 64, name="expansion_01")

# ---------------- UNet++ L2 ---------------
# X(1,1)
du_11 = Conv2DTranspose(128, (3, 3), strides = (2, 2), padding = 'same')(d3)
du_11 = concatenate([du_11,d2])
du_11 = Dropout(0.1)(du_11)
c_11 = conv2d_block(du_11, 128, name="expansion_11")

# X(0,2)
du_02 = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same')(c_11)
du_02 = concatenate([du_02,du_01,d1])
du_02 = Dropout(0.1)(du_02)
c_02 = conv2d_block(du_02, 64, name="expansion_02")

# ---------------- UNet++ L3 ---------------
# X(2,1)
du_21 = Conv2DTranspose(256, (3, 3), strides = (2, 2), padding = 'same')(d4)
du_21 = concatenate([du_21,d3])
du_21 = Dropout(0.1)(du_21)
c_21 = conv2d_block(du_21, 256, name="expansion_21")

# X(1,2)
du_12 = Conv2DTranspose(128, (3, 3), strides = (2, 2), padding = 'same')(c_21)
du_12 = concatenate([du_12,c_11,d2])
du_12 = Dropout(0.1)(du_12)
c_12 = conv2d_block(du_12, 128, name="expansion_12")

# X(0,3)
du_03 = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same')(c_12)
du_03 = concatenate([du_03,c_02,c_01,d1])
du_03 = Dropout(0.1)(du_03)
c_03 = conv2d_block(du_03, 64, name="expansion_03")

# ---------------- UNet++ L4 ---------------
# X(3,1)
u_31 = Conv2DTranspose(256, (3, 3), strides = (2, 2), padding = 'same')(d5)
u_31 = concatenate([u_31,d4])
u_31 = Dropout(0.1)(u_31)
c_31 = conv2d_block(u_31, 256, name="expansion_1")

# X(2,2)
u_22 = Conv2DTranspose(256, (3, 3), strides = (2, 2), padding = 'same')(c_31)
u_22 = concatenate([u_22,c_21,d3])
u_22 = Dropout(0.1)(u_22)
c_22 = conv2d_block(u_22, 256, name="expansion_2")

# X(1,3)
u_13 = Conv2DTranspose(128, (3, 3), strides = (2, 2), padding = 'same')(c_22)
u_13 = concatenate([u_13,c_12,c_11,d2])
u_13 = Dropout(0.1)(u_13)
c_13 = conv2d_block(u_13, 128, name="expansion_3")

# X(0,4)
u_04 = Conv2DTranspose(64, (3, 3), strides = (2, 2), padding = 'same')(c_13)
u_04 = concatenate([u_04,c_03,c_02,c_01,d1])
u_04 = Dropout(0.1)(u_04)
c_04 = conv2d_block(u_04, 64, name="expansion_4")

# ------------------------------------------------------------------------
# Final concatenation
c_L = concatenate([c_01,c_02,c_03,c_04])

out = Conv2D(1, (1,1), name="output", activation='sigmoid')(c_L)

unetpp = Model( inp, out )
unetpp.summary()

# ------------------------------------------------------------------------

with open(path_out + 'modelsummary.txt', 'w') as f:
    with redirect_stdout(f):
        unetpp.summary()

unetpp.compile(optimizer=optimizers.Adam(lr=1e-3), 
             loss=loss_func, metrics=['acc', dice_coef])


model_checkpoint  = ModelCheckpoint(path_out + 'model_best_checkpoint.h5', save_best_only=True, 
                                    monitor='val_loss', mode='min', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min')
reduceLR = ReduceLROnPlateau(patience=4, verbose=2, monitor='val_loss',min_lr=1e-4, mode='min')

callback_list = [early_stopping, reduceLR, model_checkpoint]

hist = unetpp.fit(X_train1,Y_train1,batch_size=8,epochs=100,
               validation_data=(X_val,Y_val),verbose=1)


# Calculating and drawing training plots
acc = hist.history['acc']
val_acc = hist.history['val_acc']
loss = hist.history['loss']
val_loss = hist.history['val_loss']
dice = hist.history['dice_coef']
val_dice = hist.history['val_dice_coef']

avg_acc = np.mean(acc) * 100
avg_val_acc = np.mean(val_acc) * 100
avg_loss = np.mean(loss) * 100
avg_val_loss = np.mean(val_loss) * 100
avg_dcc = np.mean(dice) * 100
avg_val_dcc = np.mean(val_dice) * 100

file0 = open(path_out + 'avg_eval.txt','w')#append mode 
file0.write("Average Training Accuracy: " + str(avg_acc)) 
file0.write("\n")
file0.write("Average Validation Accuracy: " + str(avg_val_acc)) 
file0.write("\n")
file0.write("Average Training DCC: " + str(avg_dcc)) 
file0.write("\n")
file0.write("Average Validation DCC: " + str(avg_val_dcc)) 
file0.write("\n")
file0.write("Average Training Loss: " + str(avg_loss)) 
file0.write("\n")
file0.write("Average Validation Loss: " + str(avg_val_loss)) 
file0.close()

np.savetxt(path_out + 'train_acc.txt', acc, delimiter="\n") 
np.savetxt(path_out + 'val_acc.txt', val_acc, delimiter="\n") 
np.savetxt(path_out + 'train_loss.txt', loss, delimiter="\n") 
np.savetxt(path_out + 'val_loss.txt', val_loss, delimiter="\n") 
np.savetxt(path_out + 'dcc.txt', dice, delimiter="\n") 
np.savetxt(path_out + 'val_dcc.txt', val_dice, delimiter="\n") 


f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 4))
t = f.suptitle('Unet++ Performance in Segmenting Tumors', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)
epoch_list = hist.epoch

ax1.plot(epoch_list, hist.history['acc'], label='Train Accuracy')
ax1.plot(epoch_list, hist.history['val_acc'], label='Validation Accuracy')
ax1.set_xticks(np.arange(0, epoch_list[-1], 5))
ax1.set_ylabel('Accuracy Value');ax1.set_xlabel('Epoch');ax1.set_title('Accuracy')
ax1.legend(loc="best");ax1.grid(color='gray', linestyle='-', linewidth=0.5)

ax2.plot(epoch_list, hist.history['loss'], label='Train Loss')
ax2.plot(epoch_list, hist.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(0, epoch_list[-1], 5))
ax2.set_ylabel('Loss Value');ax2.set_xlabel('Epoch');ax2.set_title('Loss')
ax2.legend(loc="best");ax2.grid(color='gray', linestyle='-', linewidth=0.5)

ax3.plot(epoch_list, hist.history['dice_coef'], label='Train DCC')
ax3.plot(epoch_list, hist.history['val_dice_coef'], label='Validation DCC')
ax3.set_xticks(np.arange(0, epoch_list[-1], 5))
ax3.set_ylabel('DCC Value');ax3.set_xlabel('Epoch');ax3.set_title('DCC')
ax3.legend(loc="best");ax3.grid(color='gray', linestyle='-', linewidth=0.5)

plt.savefig(path_out + 'Performance_Plot.jpg')

# Prediction
THRESHOLD = 0.2
test_datagen = ImageDataGenerator()
test_datagen = test_datagen.flow(X_test, Y_test, batch_size=32)
predicted_mask = (unetpp.predict(X_test)>THRESHOLD)*1
np.save(path_out + 'predictions.npy', predicted_mask)
print(predicted_mask.shape)

test_eval = unetpp.evaluate(X_test, Y_test, verbose=0)
test_loss = test_eval[0] * 100
test_acc = test_eval[1] * 100
test_dcc = test_eval[2] * 100
print('Test accuracy: ', test_acc)
print('Test loss: ', test_loss)
print('Test dcc: ', test_dcc)
file_t = open(path_out + 'test_eval.txt','w')#append mode 
file_t.write("Test Accuracy: " + str(test_acc)) 
file_t.write("\n")
file_t.write("Test Loss: " + str(test_loss)) 
file_t.write("\n")
file_t.write("Test DCC: " + str(test_dcc)) 
file_t.close() 

# ----------------------------------------------------------------------------------------------
del c_01
del c_02
del c_03
del c_04
del d1
del d2
del d3
del d4
del d5
del p1
del p2
del p3
del p4
del du_01
del du_11
del c_11
del du_02
del du_21
del c_21
del du_12
del c_12
del u_31
del c_31
del u_22
del c_22
del u_13
del c_13
del c_L
gc.collect()

# -----------------------------------------------------------------------

pred_path = path_out + 'Prediction/'

if not os.path.exists(pred_path):
    os.makedirs(pred_path)
    print("Directory Created")
else:    
    print("Directory Already Exists") 

i = 0
for idx in range(0,X_test.shape[0]):
    i = i + 1
    imageio.imwrite(pred_path + str(i) + '_Mask.jpg', Y_test[idx])
    imageio.imwrite(pred_path + str(i) + '_PredMask.jpg', predicted_mask[idx])

# Shows predicted outputs by choosing 10 Random images
plt.figure(figsize=(8,30))
i=1;total=10
temp = np.ones_like( Y_test[0] )
t = 0
for idx in np.random.randint(0,high=X_test.shape[0],size=total):
    t = t + 1
    plt.subplot(total,3,i);i+=1
    plt.imshow(X_test[idx], cmap='gray' )
    plt.title("MRI");plt.axis('off')
    
    plt.subplot(total,3,i);i+=1
    plt.imshow(X_test[idx], cmap='gray' )
    plt.imshow(temp - Y_test[idx], alpha=0.2, cmap='Set1' )
    plt.title("Ground Truth");plt.axis('off')
    
    plt.subplot(total,3,i);i+=1
    plt.imshow(X_test[idx], cmap='gray' )
    plt.imshow(temp - predicted_mask[idx],  alpha=0.2, cmap='Set1' )
    plt.title("Predicted Tumor");plt.axis('off')

plt.savefig(path_out + 'Sample_Output_Plot.jpg')

# ----------------------------------------------------------------------------------------------

#End Time
print("--- Time : %s seconds ---" % (time.time() - start_time))
f_time = open(path_out + 'Time_Info.txt', 'w')
print("%s %f" % ("Execution Time: ", time.time() - start_time), file=f_time)
f_time.close()
